## Topic Segmentation

In [1]:
# Add project path to the PYTHONPATH

import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())

In [2]:
from nlp.dataset import Dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rmohashi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
filename = '../datasets/topic_segmentation/1175455210979725313-1175407650365935616_batman.csv'
dataset = Dataset(filename, label_col='label', text_col='text')
dataset.load()
dataset.preprocess_texts(no_emoji=True)

Time to clean up: 3.49 sec


In [4]:
df = dataset.dataframe
df.cleaned.head()

0    gonna hang tonight added batman fortnut last t...
1    bat signal projected onto landmarks across wor...
2                                    batman theme song
3    die hard batman fan ring tone tv series theme ...
4    not scared almost anything terrified joker ori...
Name: cleaned, dtype: object

In [5]:
import gensim
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/rmohashi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess_tweet(text):
    result = []
    for token in text.split():
        result.append(token)
#         result.append(lemmatize_stemming(token))
            
    return result

In [8]:
processed_tweets = [preprocess_tweet(tweet) for tweet in df.cleaned]

In [9]:
%%time
dictionary = gensim.corpora.Dictionary(processed_tweets)
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)
bow_corpus = [dictionary.doc2bow(tweets) for tweets in processed_tweets]

CPU times: user 485 ms, sys: 0 ns, total: 485 ms
Wall time: 487 ms


In [10]:
document_num = 0
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 0 ("added") appears 1 time.
Word 1 ("gonna") appears 1 time.
Word 2 ("justice") appears 1 time.
Word 3 ("last") appears 1 time.
Word 4 ("need") appears 1 time.
Word 5 ("see") appears 1 time.
Word 6 ("thing") appears 1 time.
Word 7 ("tonight") appears 1 time.


In [11]:
%%time
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 5, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 4)

CPU times: user 32.4 s, sys: 1.94 s, total: 34.3 s
Wall time: 40.6 s


In [12]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.029*"favorite" + 0.024*"today" + 0.018*"one" + 0.017*"knight" + 0.016*"dark" + 0.016*"movie" + 0.015*"comics" + 0.014*"comic" + 0.013*"th" + 0.013*"time"


Topic: 1 
Words: 0.116*"fortnite" + 0.043*"x" + 0.030*"new" + 0.028*"gotham" + 0.024*"event" + 0.023*"city" + 0.022*"arkham" + 0.021*"live" + 0.020*"code" + 0.019*"batman:"


Topic: 2 
Words: 0.061*"best" + 0.050*"bat" + 0.044*"celebrate" + 0.032*"world" + 0.032*"signal" + 0.024*"ever" + 0.021*"dccomics" + 0.020*"across" + 0.020*"projected" + 0.020*"landmarks"


Topic: 3 
Words: 0.022*"game" + 0.018*"like" + 0.014*"good" + 0.014*"fortnite" + 0.011*"gonna" + 0.011*"would" + 0.011*"skin" + 0.011*"knight" + 0.010*"everyone" + 0.010*"no"


Topic: 4 
Words: 0.023*"like" + 0.020*"skin" + 0.018*"bundle" + 0.017*"get" + 0.015*"pack" + 0.015*"want" + 0.015*"buy" + 0.014*"really" + 0.013*"joker" + 0.012*"need"




In [13]:
import pyLDAvis.gensim

In [14]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

/home/rmohashi/anaconda3/envs/lda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.099383  0.105457       1        1  29.120562
3     -0.042112  0.136078       2        1  22.675056
4     -0.083556  0.112606       3        1  18.888880
2      0.269950 -0.158939       4        1  14.699130
1     -0.243666 -0.195202       5        1  14.616370, topic_info=    Category         Freq             Term        Total  loglift  logprob
85   Default  1124.000000         fortnite  1124.000000  30.0000  30.0000
91   Default   480.000000             best   480.000000  29.0000  29.0000
9    Default   392.000000              bat   392.000000  28.0000  28.0000
329  Default   330.000000                x   330.000000  27.0000  27.0000
10   Default   363.000000        celebrate   363.000000  26.0000  26.0000
15   Default   252.000000            world   252.000000  25.0000  25.0000
14   Default   258.000000           signal   258.000000  24.0000  24.0000
303  Default   466.000000         favorite   466.000000  23.0000  23.0000
369  Default   302.000000           gotham   302.000000  22.0000  22.0000
101  Default   188.000000            event   188.000000  21.0000  21.0000
87   Default   382.000000              new   382.000000  20.0000  20.0000
470  Default   249.000000             ever   249.000000  19.0000  19.0000
368  Default   206.000000             city   206.000000  18.0000  18.0000
8    Default   156.000000           across   156.000000  17.0000  17.0000
13   Default   153.000000        projected   153.000000  16.0000  16.0000
11   Default   153.000000        landmarks   153.000000  15.0000  15.0000
12   Default   152.000000             onto   152.000000  14.0000  14.0000
45   Default   353.000000           comics   353.000000  13.0000  13.0000
198  Default   203.000000         dccomics   203.000000  12.0000  12.0000
237  Default   313.000000             game   313.000000  11.0000  11.0000
139  Default   177.000000             code   177.000000  10.0000  10.0000
782  Default   137.000000          players   137.000000   9.0000   9.0000
458  Default   205.000000             live   205.000000   8.0000   8.0000
203  Default   270.000000           arkham   270.000000   7.0000   7.0000
427  Default   229.000000           bundle   229.000000   6.0000   6.0000
326  Default   373.000000             skin   373.000000   5.0000   5.0000
256  Default   498.000000            today   498.000000   4.0000   4.0000
187  Default   208.000000              use   208.000000   3.0000   3.0000
76   Default   576.000000             like   576.000000   2.0000   2.0000
144  Default   170.000000             pack   170.000000   1.0000   1.0000
..       ...          ...              ...          ...      ...      ...
109   Topic5    18.023302         announce    18.839186   1.8788  -6.0581
835   Topic5    19.760191         download    20.769356   1.8732  -5.9661
806   Topic5    16.309799         official    17.158802   1.8723  -6.1580
843   Topic5    13.306022         becoming    14.237028   1.8554  -6.3616
828   Topic5    16.451540           saving    17.653929   1.8525  -6.1494
568   Topic5    96.232399            later   104.982819   1.8360  -4.3830
300   Topic5    82.164665        crossover    89.784248   1.8343  -4.5411
700   Topic5     8.163646        lifetime'     8.972820   1.8285  -6.8501
785   Topic5    10.519769           gordon    12.058942   1.7865  -6.5965
747   Topic5    26.031353              tie    29.946995   1.7829  -5.6905
306   Topic5    12.894086         episodes    14.892790   1.7789  -6.3930
139   Topic5   151.151443             code   177.483154   1.7624  -3.9315
368   Topic5   174.109222             city   206.043976   1.7546  -3.7901
85    Topic5   892.830200         fortnite  1124.906738   1.6920  -2.1554
388   Topic5    94.772400        available   114.055679   1.7378  -4.3983
458   Topic5   162.861496             live   205.631714   1.6898  -3.8569
369   Topic5   217.991272  

In [15]:
df[df.text.str.contains('amp')].text

11      @beefkr10z Quite hard to tell currently what w...
14      @MemeBloody Isn’t twitter for expressing your ...
78      Celebrating 80 Years of #Batman with #BatmanDa...
147     Batman Jones of course, Batman saved a couple ...
149     Batman is drawn into conflict with Mr. Whisper...
154     Pow! #Batman was born in #TheBronx. Writer Bil...
175     &amp; have psychological damage/abnormalities:...
177     @AstroIoxy Wow!!! Thats what happens when you ...
181     I’ve been fortunate to have played in the Batm...
189     I see your smoke bomb Batman &amp; I'll raise ...
191     Batman, Batman Returns, The Dark Knight Triolo...
205     Lastly, I watched Batman Forever and Batman &a...
228     ⚡️ x1 Batman Caped Crusader Pack Giveaway ⚡️ \...
262     If I was off today for #BatmanDay I would've d...
267     🐾Catwoman/Selina Kyle\n🐾Expert Cat Burglar\n🐾B...
312     Happy #BatmanDay! The Caped Crusader is 80 yrs...
352     https://t.co/dpIbGyyPPH Crime-fighting duo #Ba...
433     Happy 